In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
import json
import unidecode
import math
import numpy as np

In [2]:
atri_jugadores_19 = pd.read_csv('atributos_jugadores_2019.csv')
atri_jugadores_20 = pd.read_csv('atributos_jugadores_2020.csv')
atri_jugadores_21 = pd.read_csv('atributos_jugadores_valor_2021.csv')
atri_jugadores_22 = pd.read_csv('atributos_jugadores_21_22.csv')
atri_jugadores_23 = pd.read_csv('atributos_jugadores_valor_2023.csv')

In [3]:
atri_jugadores_19.rename(columns={'reflejos':'reflejo'},inplace=True)
atri_jugadores_22.rename(columns={'reflejos':'reflejo'},inplace=True)
atri_jugadores_20.rename(columns={'reflejos':'reflejo'},inplace=True)

In [4]:
atri_jugadores_total = pd.concat([atri_jugadores_19,atri_jugadores_20,atri_jugadores_21,atri_jugadores_22,atri_jugadores_23],axis=0,ignore_index=True)
atri_jugadores_total.drop_duplicates(subset=['nombre'], keep='first',inplace=True)

In [5]:
# elimino valor_en_M ya que la tabla de 2022 no incluia este dato.
atri_jugadores_total = atri_jugadores_total.drop(['Unnamed: 0','valor_en_M'],axis=1)

In [6]:
#Jugadores que aparecian en loa enfrentamientos pero no en las plantillas de los equipos 

jugadores_nuevos = pd.DataFrame({"nombre":['JESUS OWONO','A. PRIMO','OIER','PAU LOPEZ'],"equipo":['ALAVES','LEVANTE','ESPANYOL','REAL BETIS'],"posicion":['PORTERO','PORTERO','PORTERO','PORTERO'],"ritmo":[np.nan,np.nan,np.nan,np.nan],"tiro":[np.nan,np.nan,np.nan,np.nan],
                            "pase":[np.nan,np.nan,np.nan,np.nan],"regate":[np.nan,np.nan,np.nan,np.nan],"defensa":[np.nan,np.nan,np.nan,np.nan],"fisico":[np.nan,np.nan,np.nan,np.nan],"salto":[73,67,66,71],"estirada":[81,72,74,80],"parada":[79,63,72,81],"saque":[71,71,72,72],"colocacion":[83,66,75,81],"reflejo":[84,71,71,83]},)

atri_jugadores_total = pd.concat([atri_jugadores_total,jugadores_nuevos],axis=0)

In [7]:
atri_jugadores_total['nombre'].replace('A. PRIMO','ALEJANDRO PRIMO',inplace=True)

In [14]:
#eliminimo jugadores duplicados por su nombre, equipo y posicion
atri_jugadores_total.drop_duplicates(subset=['nombre'],inplace=True) 

In [15]:
fecha = []
fecha_atributos = []
alineacion_l = []
local = []
gol_l = []
resultado_l = []
titulares_l = []
portero_l = []
defensas_l = []
centrocampistas_l = []
delanteros_l = []
alineacion_v = []
visitante = []
gol_v = []
resultado_v = []
titulares_v = []
portero_v = []
defensas_v = []
centrocampistas_v = []
delanteros_v = []
victorias_l = []
empates = []
victorias_v = []
resultado_local_atributos = []
victoria_local = []
empates_equipos = []
victoria_visitante = []

In [16]:

# Enfrentamientos y datos

    
for i in range(23,24):
    
    r = requests.get(f'https://es.besoccer.com/competicion/resultados/primera/2023/grupoall/jornada{i}') # optener html de la pagina
    soup = bs(r.content,'html.parser')
    link_alineacion = soup.find('div',attrs={"class":"match-list-new"}).find_all('a')

    for link in link_alineacion:
        r1 = requests.get(f"{link['href']}/alineaciones") # optener html de la pagina
        soup1 = bs(r1.content,'html.parser')
        datos = soup1.find('main',attrs={"class":"content"})

        fecha_partido = datos.find_all('div')[0].div['starttime'][0:10]
        fecha.append(fecha_partido)
        gol_local = soup1.find('span',attrs={"class":"r1"}).text
        gol_visitante = soup1.find('span',attrs={"class":"r2"}).text

        # Resultado
        def resultado(gol_local,gol_visitante):

            if gol_local>gol_visitante:
                res_local = 'G'
                res_vis = 'P'
                return res_local, res_vis
            elif gol_local<gol_visitante:
                res_local = 'P'
                res_vis = 'G'
                return res_local, res_vis
            else:
                res_local = 'E'
                res_vis = 'E'
                return res_local, res_vis

        res_local,res_vis=resultado(gol_local,gol_visitante)
        
        resultado_l.append(res_local)
        resultado_v.append(res_vis)

        # historial entre equipos
        r2 = requests.get(f"{link['href']}/previa") # optener html de la pagina
        soup2 = bs(r2.content,'html.parser')
        ultimos_enfren = soup2.find_all('p',attrs={"class":"big-num"})

        try:
            vic_l = ultimos_enfren[0].text
        except:
            vic = 0  
        try: 
            empa  = ultimos_enfren[1].text
        except:
            empa = 0
        try:
            vic_vis = ultimos_enfren[2].text
        except:
            vic_vis = 0
        
        # EQUIPO LOCAL
        try:
            alineacion_l.append(datos.find_all('ul')[0]['data-tacticname'])
        except:
            alineacion_l.append(np.nan)
        try:
            local.append(unidecode.unidecode(datos.find_all('a')[1].text).upper()) 
        except:
            local.append(np.nan)
        gol_l.append(gol_local)
        
        # titulares 
        titular_l = soup1.find('ul',attrs={"class":"lineup local"}).find_all('a')
        titular_partido_l = [] # Listas para sacar el promedio de posiciones por partido
        resultado_local_tabla = []
        for i in titular_l:
            titulares_l.append(unidecode.unidecode(i.img['alt']).upper()) 
            titular_partido_l.append(unidecode.unidecode(i.img['alt']).upper()) 
            fecha_atributos.append(fecha_partido) # fecha para tabla de atributos
            resultado_local_atributos.append(res_local)
            resultado_local_tabla.append(res_local)
            try:
                victoria_local.append(vic_l)
            except:
                victoria_local.append(0)
            try:
                empates_equipos.append(empa)
            except:
                empates_equipos.append(0)
            try:
                victoria_visitante.append(vic_vis)
            except:
                victoria_visitante.append(0)
    

        # atributos 
        jug_l = pd.DataFrame({'fecha':fecha_partido,'resultado':resultado_local_tabla,'nombre':titular_partido_l})
        media_atri_jug_l = pd.merge(jug_l,atri_jugadores_total,how='left') # hago un merge entre los titulares y el dataframe de atributos
        media_posi_l=media_atri_jug_l.round(decimals=2).groupby(['posicion']).mean(numeric_only=True) # los agrupo por posicion y creo una media de los atributos
        try:
            portero_l.append(media_posi_l.mean(axis=1)[3]) # hago una media general por posicion y  obtengo el dato  de cada posicion
        except:
            portero_l.append(72.01620370370371) #media genereal de los porteros
        try:
            defensas_l.append(media_posi_l.mean(axis=1)[1])
        except:
            defensas_l.append(62.10348842258565)
        try:
            centrocampistas_l.append(media_posi_l.mean(axis=1)[0])
        except:
            centrocampistas_l.append(66.23712642609128)
        try:
            delanteros_l.append(media_posi_l.mean(axis=1)[2])
        except:
            delanteros_l.append(65.85402140022676)

        
        # EQUIPO VISITANTE
        try:
            alineacion_v.append(datos.find_all('ul')[1]['data-tacticname'])
        except:
            alineacion_v.append(np.nan)
        try:
            visitante.append(unidecode.unidecode(datos.find_all('a')[4].text).upper())
        except:
            visitante.append(np.nan)
        gol_v.append(gol_visitante)

        # titulares 
        titular_v = soup1.find('ul',attrs={"class":"lineup visitor"}).find_all('a')
        titular_partido_v = [] # Listas para sacar el promedio de posiciones por partido
        for i in titular_v:
            titulares_v.append(unidecode.unidecode(i.img['alt']).upper()) 
            titular_partido_v.append(unidecode.unidecode(i.img['alt']).upper())

        # atributos 
        jug_v = pd.DataFrame({'fecha':fecha_partido,'nombre':titular_partido_v})
        media_atri_jug_v = pd.merge(jug_v,atri_jugadores_total,how='left') 
        media_posi_v=media_atri_jug_v.round(decimals=2).groupby(['posicion']).mean(numeric_only=True) 
        try:
            portero_v.append(media_posi_v.mean(axis=1)[3])
        except:
            portero_v.append(72.01620370370371)
        try:
            defensas_v.append(media_posi_v.mean(axis=1)[1])
        except:
            defensas_v.append(62.10348842258565)
        try:
            centrocampistas_v.append(media_posi_v.mean(axis=1)[0])
        except:
            centrocampistas_v.append(66.23712642609128)
        try:
            delanteros_v.append(media_posi_v.mean(axis=1)[2])
        except:
            delanteros_v.append(65.85402140022676)
        
        

        try:
            victorias_l.append(vic_l)
        except:
            victorias_l.append(0)
        try:
            empates.append(empa)
        except:
            empates.append(0)
        try:
            victorias_v.append(vic_vis)
        except:
            victorias_v.append(0)
    


In [172]:
liga_21_22 = pd.DataFrame({'fecha':fecha,'portero_l':portero_l,'defensas_l':defensas_l,'centrocampistas_l':centrocampistas_l,'delanteros_l':delanteros_l,'alineacion_l':alineacion_l,'local':local,'Resultado_l':resultado_l,'gol_l':gol_l,
                                            'gol_v':gol_v,'Resultado_v':resultado_v,'visitante':visitante,'alineacion_v':alineacion_v,'portero_v':portero_v,'defensas_v':defensas_v,'centrocampistas_v':centrocampistas_v,'delanteros_v':delanteros_v,'victorias_l':victorias_l,'empates':empates,'victorias_v':victorias_v})

In [173]:
liga_21_22

,fecha,portero_l,defensas_l,centrocampistas_l,delanteros_l,alineacion_l,local,Resultado_l,gol_l,gol_v,Resultado_v,visitante,alineacion_v,portero_v,defensas_v,centrocampistas_v,delanteros_v,victorias_l,empates,victorias_v
0,2019-08-16,77.166667,68.200000,70.722222,46.583333,4-2-3-1,ATHLETIC,G,1,0,P,BARCELONA,4-3-3,86.333333,68.777778,71.944444,76.111111,75,38,114
1,2019-08-17,73.166667,62.000000,68.666667,73.333333,4-4-2,CELTA,P,1,3,G,REAL MADRID,4-3-3,81.500000,71.666667,73.666667,74.666667,34,20,68
2,2019-08-17,79.000000,67.250000,73.000000,72.333333,4-4-2,VALENCIA,E,1,1,E,R. SOCIEDAD,4-2-3-1,74.000000,62.375000,74.333333,69.666667,67,36,49
3,2019-08-17,68.166667,58.583333,63.500000,56.000000,4-5-1,MALLORCA,G,2,1,P,EIBAR,4-4-2,75.000000,61.222222,66.083333,64.625000,8,3,3
4,2019-08-17,72.016204,59.166667,62.916667,67.166667,3-5-2,LEGANES,P,0,1,G,OSASUNA,4-5-1,72.666667,61.375000,66.916667,68.250000,7,4,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2020-07-19,71.666667,61.333333,67.463333,68.722222,4-4-2,ESPANYOL,E,0,0,E,CELTA,4-3-3,66.000000,62.958333,67.250000,72.250000,57,32,43
376,2020-07-19,72.016204,61.500000,65.166667,69.722222,4-4-2,LEVANTE,G,1,0,P,GETAFE,4-4-2,77.000000,68.791667,64.277778,65.000000,12,15,11
377,2020-07-19,76.833333,59.416667,65.500000,64.388889,4-2-3-1,GRANADA,G,4,0,P,ATHLETIC,4-2-3-1,73.666667,66.766667,61.916667,49.888889,13,12,28
378,2020-07-19,72.666667,63.166667,68.500000,63.888889,4-4-2,OSASUNA,E,2,2,E,MALLORCA,4-4-2,72.016204,61.486111,64.583333,56.000000,21,21,21


In [18]:
liga_22_23_fecha_23 = pd.DataFrame({'portero_l':portero_l,'defensas_l':defensas_l,'centrocampistas_l':centrocampistas_l,'delanteros_l':delanteros_l,'alineacion_l':alineacion_l,'local':local,'Resultado_l':resultado_l,
                                            'visitante':visitante,'alineacion_v':alineacion_v,'portero_v':portero_v,'defensas_v':defensas_v,'centrocampistas_v':centrocampistas_v,'delanteros_v':delanteros_v,'victorias_l':victorias_l,'empates':empates,'victorias_v':victorias_v})

In [13]:
liga_22_23_fecha_23.to_csv('liga_22_23_fecha_23.csv')

In [177]:
jugadores_l = pd.DataFrame({'fecha':fecha_atributos,'resultado':resultado_local_atributos,'victoria_local':victoria_local,'empates':empates_equipos,'victoria_visitante':victoria_visitante,'nombre':titulares_l}) 
atributos_detallados_l = pd.merge(jugadores_l,atri_jugadores_total,left_on='nombre', right_on='nombre', how='left')
jugadores_v = pd.DataFrame({'fecha':fecha_atributos,'nombre':titulares_v}) 
atributos_detallados_v = pd.merge(jugadores_v,atri_jugadores_total,left_on='nombre', right_on='nombre', how='left')
atributos_por_partido = pd.merge(atributos_detallados_l,atributos_detallados_v,left_index=True,right_index=True)

In [178]:
atributos_por_partido

,fecha_x,resultado,victoria_local,empates,victoria_visitante,nombre_x,equipo_x,posicion_x,ritmo_x,tiro_x,...,pase_y,regate_y,defensa_y,fisico_y,salto_y,estirada_y,parada_y,saque_y,colocacion_y,reflejo_y
0,2019-08-16,G,75,38,114,UNAI SIMON,ATHLETIC CLUB,PORTERO,NaN,NaN,...,NaN,NaN,NaN,NaN,79.0,88.0,85.0,88.0,88.0,90.0
1,2019-08-16,G,75,38,114,UNAI NUNEZ,ATHLETIC CLUB,DEFENSA,73.0,25.0,...,66.0,67.0,61.0,77.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-08-16,G,75,38,114,YERAY,ATHLETIC CLUB,DEFENSA,69.0,55.0,...,70.0,63.0,59.0,75.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-08-16,G,75,38,114,YURI,ATHLETIC CLUB,DEFENSA,83.0,71.0,...,81.0,86.0,68.0,80.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-08-16,G,75,38,114,ANDER CAPA,ATHLETIC CLUB,DEFENSA,83.0,68.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4175,2020-07-19,E,76,34,47,H. HERRERA,ATLETICO DE MADRID,CENTROCAMPISTA,72.0,71.0,...,67.0,71.0,60.0,70.0,NaN,NaN,NaN,NaN,NaN,NaN
4176,2020-07-19,E,76,34,47,VITOLO,GETAFE,DELANTERO,76.0,73.0,...,74.0,71.0,65.0,72.0,NaN,NaN,NaN,NaN,NaN,NaN
4177,2020-07-19,E,76,34,47,KOKE,ATLETICO DE MADRID,CENTROCAMPISTA,67.0,76.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4178,2020-07-19,E,76,34,47,A. MORATA,ATLETICO DE MADRID,DELANTERO,82.0,70.0,...,80.0,84.0,43.0,75.0,NaN,NaN,NaN,NaN,NaN,NaN


In [179]:
#atributos_por_partido.round(decimals=2).to_csv('#3_atributos_jugadores_por_partido_liga_20_21.csv')

In [180]:
#liga_21_22.round(decimals=2).to_csv('liga_20_21_besoccer.csv')